In [1]:
import numpy as np 
import pygame 
import math
import sys 
from copy import deepcopy 
import time 
import random 
import os
import threading

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from Ataxx.Ataxx.AtaxxGame import AtaxxGame
import Ataxx.Ataxx.AtaxxPlayers as APlayers

from Go.Go.GoGame import GoGame
import Go.Go.GoPlayers as GPlayers

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#Funções para obter os jogadores de cada jogo

def get_ataxx_players(game):
    hp = APlayers.HumanPlayer(game)
    gp = APlayers.GreedyPlayer(game)
    A4x4p = APlayers.A4x4(game)
    A5x5p = APlayers.A5x5(game)
    A6x6p = APlayers.A6x6(game)
    AFlexp = APlayers.AFlex(game)

    return hp, gp, A4x4p, A5x5p, A6x6p, AFlexp


def get_go_players(game):
    hp = GPlayers.HumanPlayer(game)
    gp = GPlayers.GreedyPlayer(game)
    G7x7p = GPlayers.G7x7(game)
    G9x9p = GPlayers.G9x9(game)
    
    return hp, gp, G7x7p, G9x9p

In [4]:
SQUARE_SIZE = 75 

PLAYER1 = 1
PLAYER2 = -1

BLUE = (0,0,255) 
RED = (255,0,0) 
WHITE_COLOR = (255,255,255) 
BLACK_COLOR = (0,0,0) 
BACKGROUND_COLOR = (240, 196, 52)


In [5]:
import socket
import random
import time

def get_action(player, board, curPlayer, screen):
    """
    Função para obter a jogada
    """
    return player.play(board, curPlayer, screen)

In [7]:
def connect_to_server(host='localhost', port=12345):
    """
    Função para conectar ao servidor
    """
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    SIZE = int(Game[-1])
    
    if Game[0] == "A": # Se o jogo for o ataxx
        game = AtaxxGame(SIZE)
        hp, gp, A4x4p, A5x5p, A6x6p, AFlexp = get_ataxx_players(game)
        player = None # Para escolher o jogador manualmente basta trocar o None pelo jogador pretendido
        if player is None: # Caso o jogador seja None, ele é escolhido tendo em conta o jogo e o tamanho do tabuleiro
            if "Fx" in Game:
                player = AFlexp
            else:
                if SIZE == 4:
                    player = A4x4p
                elif SIZE == 5:
                    player = A5x5p
                elif SIZE == 6:
                    player = A6x6p

    elif Game[0] == "G": # Se o jogo for o ataxx 
        
        game = GoGame(SIZE)
        hp, gp, G7x7p, G9x9p = get_go_players(game)
        player = None
        
        if player is None:
            if SIZE == 7:
                player = G7x7p
            elif SIZE == 9:
                player = G9x9p
        
    print(player)
    board = game.getInitBoard()
    
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    width = SIZE * SQUARE_SIZE 
    height = SIZE * SQUARE_SIZE 
    screen_size = (width, height) 
    
    if "1" in response:
        ag=1
    else:
        ag=2
        
    first=True
    curPlayer = 1
    invalid_counter = 0
    is_pass = False # Flag para ver se o jogo vai passar a jogada por ter feito 3 jogadas inválidas

    screen = pygame.display.set_mode(screen_size)   
    if Game[0] == 'G': # Se o jogo for go pinta o fundo da interface (puramente estético)
        screen.fill(BACKGROUND_COLOR)

    pygame.display.set_caption(f'Agent {ag}')
    game.display(board, screen)
    time.sleep(10)
    
    while True:
        pygame.event.pump()
        
        if Game[0] == "A": # Se o jogo for o ataxx
            if (ag == 1 or not first) and not is_pass:
                if invalid_counter >= 3: # Se já tiver feito 3 jogadas inválidas
                    is_pass = True
                    invalid_counter = 0
                    curPlayer = -curPlayer
                else:
                    if game.getGameEnded(board, curPlayer) is None: # Se o jogo ainda não acabou
                        action = get_action(player, game.getCanonicalForm(board, curPlayer), curPlayer, screen)
                        if action is None:
                            is_pass = True
                            invalid_counter = 0
                        else:
                            if type(action) is int: # Se a ação for um número inteiro calcula o movimento correspondente a esse número
                                new_piece = (int(action/SIZE), action%SIZE)
                                old_piece = game.get_piece_to_move(board, new_piece, curPlayer)
                                (x1, y1), (x2, y2) = old_piece, new_piece
                            else: 
                                (x1, y1, x2, y2) = action

                            move_msg = f"MOVE {x1},{y1},{x2},{y2}"
                            time.sleep(1)
                            client_socket.send(move_msg.encode())
                            print("Send:",move_msg)

                    # Wait for server response
                    response = client_socket.recv(1024).decode()
                    print(f"Server Response1: {response}")
                    if "END" in response: break
                    if response == "NOT YOU":
                        print("ok")
                        curPlayer = -curPlayer
                        #continue
                    elif response == "VALID":
                        move = (x1, y1, x2, y2)
                        board, curPlayer = game.getNextState(board, curPlayer, move)
                        game.display(board, screen)
                        invalid_counter = 0
                    elif response == "INVALID":
                        invalid_counter += 1
                        continue

            first=False
            response = client_socket.recv(1024).decode()
            print(f"Server Response2: {response}")

            if response == "YOU": # Se a resposta for "YOU", quer dizer que é a vez deste agente jogar após o outro ter perdido a vez
                curPlayer = -curPlayer
                continue

            if "END" in response: break

            oppx1, oppy1 , oppx2, oppy2 = int(response[5]), int(response[7]), int(response[9]), int(response[11])
            board, curPlayer = game.getNextState(board, curPlayer, (oppx1, oppy1, oppx2, oppy2))
            game.display(board, screen)
            is_pass = False

        elif Game[0] == "G": # Se o jogo for o go (semelhante ao processo do ataxx)
            
            if (ag == 1 or not first) and not is_pass:
                if invalid_counter >= 3:
                    is_pass = True
                    invalid_counter = 0
                    curPlayer = -curPlayer
                else:

                    if game.getGameEnded(board, curPlayer) is None:
                        action = get_action(player, game.getCanonicalForm(board, curPlayer), curPlayer, screen)
                        if action is None:
                            is_pass = True
                            invalid_counter = 0
                        else:
                            if action == game.size*game.size:
                                move = (9, 9)
                            else:
                                move = (int(action/SIZE), action%SIZE)
                            move_msg = f"MOVE {move[0]},{move[1]}"
                            time.sleep(1)
                            client_socket.send(move_msg.encode())
                            print("Send:",move_msg)

                    # Wait for server response
                    response = client_socket.recv(1024).decode()
                    print(f"Server Response1: {response}")
                    if "END" in response: break
                    if response == "NOT YOU":
                        curPlayer = -curPlayer
                        #continue
                    elif response == "VALID":
                        board, curPlayer = game.getNextState(board, curPlayer, move)
                        game.display(board, screen)
                        invalid_counter = 0
                    elif response == "INVALID":
                        invalid_counter += 1
                        continue

            first=False
            response = client_socket.recv(1024).decode()
            print(f"Server Response2: {response}")

            if response == "YOU":
                curPlayer = -curPlayer
                continue

            if "END" in response: break

            oppx, oppy  = int(response[5]), int(response[7])
            board, curPlayer = game.getNextState(board, curPlayer, (oppx, oppy))
            game.display(board, screen)
            is_pass = False
            
    game_over = True
    while game_over: # A interface continua aberta até ser fechada clicando no botão para fechar
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:
                pygame.quit()
                client_socket.close()
                game_over = False
                
if __name__ == "__main__":
    pygame.init()
    game = None
    connect_to_server()

Server ResponseINIT: AG1 G9x9
Playing: G9x9
Send: MOVE 0,1
Server Response1: VALID
Server Response2: MOVE 0,4
Send: MOVE 5,5
Server Response1: VALID
Server Response2: MOVE 6,2
Send: MOVE 7,7
Server Response1: VALID
Server Response2: MOVE 6,3
Send: MOVE 1,2
Server Response1: VALID
Server Response2: MOVE 8,3
Send: MOVE 9,9
Server Response1: VALID
Server Response2: MOVE 9,9
Server Response1: END -1 5 9.5
